Define file path

In [2]:
import pandas as pd
import os
import csv

# Define input and output file paths
path = os.path.join(os.path.dirname(os.getcwd()))
path='/Users/ayla/workflow'
ipath = os.path.join(path,'data')
opath = os.path.join(path,'data')
input_file_dir=os.path.join(ipath)
output_file_dir=os.path.join(opath)


Note that the ipath and opath are identical. This is because this script uses one file called "data" for all inputs and outputs. 

# Further Clean the Data
The VRE table will include some BGs that have zero population. We must remove these zero population BGs from our csv file. 

Before we do this, we should identify the number of BGs that have a non-zero population but have zero population 2.0 times the poverty line (this implies that the entire population is low income).
We don’t want to accidentally delete BGs that are 100% low income. The script below identifies 100% low income BGs. It also retruns the total number of rows in the input file. This will be relevant later.


In [115]:
#Identifies row pairs for which both order = 8 has a corresponding estimate that equals zero (indicating zero population 2.0 times poverty line) 
#AND order = 1 has a non-zero estimate (has a total population)
#If a row pair meets these requirements, it means it is a 100% low income block group
import pandas as pd
#'/Users/ayla/workflow/data/edited_VRE_complete_NJ.csv'

# Load the CSV file
input_file = f"{ipath}/{'edited_VRE_complete_NJ.csv'}"  # Replace with your CSV file name
data = pd.read_csv(input_file)

# Calculate the number of block groups in the input file
num_original = data.shape[0] // 2
print("Number of block groups in the input file:", num_original)

# Initialize a list to store the identified pairs
identified_pairs = []

# Iterate over the dataframe, processing each pair of rows
for i in range(1, data.shape[0], 2):
    if data.iloc[i-1, 1] == 1 and data.iloc[i, 1] == 8 and data.iloc[i-1, 0] == data.iloc[i, 0]:
        # Extract the GEOID for the current pair
        geoid = data.iloc[i-1, 0]
        
        # Extract the estimates for the current pair
        estimate_order_1 = data.iloc[i-1]['ESTIMATE']
        estimate_order_8 = data.iloc[i]['ESTIMATE']
        
        # Check if ORDER = 8 estimate is zero and ORDER = 1 estimate is non-zero
        if estimate_order_8 == 0 and estimate_order_1 != 0:
            # Append the rows to the identified_pairs list
            identified_pairs.append(data.iloc[i-1, :])
            identified_pairs.append(data.iloc[i, :])

# Convert the list of identified pairs to a DataFrame for easier visualization
all_low_income_bg_df = pd.DataFrame(identified_pairs)

#define df as data frame of identified pairs only
df = pd.DataFrame(identified_pairs)

#Print number of identified pairs
num_rows = df.shape[0]
print("Number of 100% low income block groups")
#print "num_rows" divided by two since each pair of rows represents 1 block group
print(num_rows/2)
#store this value for later 
num_all_low_income = (num_rows/2)
# Print the identified pairs
print(all_low_income_bg_df)

# Define the output file path
output_file = f"{opath}/100%_low_income_bgs.csv"

# Write the identified pairs to a CSV file
df.to_csv(output_file, index=False)

print(f"Results have been written to {output_file}")

Number of block groups in the input file: 6599
Number of 100% low income block groups
11.0
                      GEOID  ORDER  ESTIMATE  Var_Rep1  Var_Rep2  Var_Rep3  \
3260  1500000US340076105001      1       147       109       135       186   
3261  1500000US340076105001      8         0         0         0         0   
3326  1500000US340076115003      1        78        99        79       106   
3327  1500000US340076115003      8         0         0         0         0   
5104  1500000US340139801001      1         4         4         4         5   
5105  1500000US340139801001      8         0         0         0         0   
5138  1500000US340155002043      1        54        40        48        21   
5139  1500000US340155002043      8         0         0         0         0   
5496  1500000US340155020026      1        11        10         1        24   
5497  1500000US340155020026      8         0         0         0         0   
5522  1500000US340155024001      1         4       

Now let's consider the BGs that have zero total population. We know logically that these BGs should also contain zero individuals 2.0 times the poverty line. 

However, we must test this assumption to ensure the accuracy and validity of our data. In order to perform this test, run the following script that tests for BG pairs in which total population is zero but percent 2.0 times the poverty line is non-zero. 


In [116]:
#order = 1 (total population) has an estimate that equals zero, but order = 8 (population > 2.0 times the poverty line) has a non-zero estimate

import pandas as pd

# Load the CSV file (same input file as previous code box)
input_file = f"{ipath}/{'edited_VRE_complete_NJ.csv'}"  # Replace with your CSV file name
data = pd.read_csv(input_file)

# Initialize a list to store the identified pairs
identified_pairs = []

# Iterate over the dataframe, processing each pair of rows
for i in range(1, data.shape[0], 2):
    if data.iloc[i-1, 1] == 1 and data.iloc[i, 1] == 8 and data.iloc[i-1, 0] == data.iloc[i, 0]:
        # Extract the GEOID for the current pair
        geoid = data.iloc[i-1, 0]
        
        # Extract the estimates for the current pair
        estimate_order_1 = data.iloc[i-1]['ESTIMATE']
        estimate_order_8 = data.iloc[i]['ESTIMATE']
        
        # Check if ORDER = 1 estimate is zero and ORDER = 8 estimate is non-zero
        if estimate_order_1 == 0 and estimate_order_8 != 0:
            # Append the rows to the identified_pairs list
            identified_pairs.append(data.iloc[i-1, :5])
            identified_pairs.append(data.iloc[i, :5])

# Convert the list of identified pairs to a DataFrame for easier visualization
zero_pop_nonzero_est_df = pd.DataFrame(identified_pairs)

# Print the identified pairs
print("Row pairs for which total population estimate is zero and population > 2.0 poverty line estimate is non-zero:")
print(zero_pop_nonzero_est_df)

Row pairs for which total population estimate is zero and population > 2.0 poverty line estimate is non-zero:
Empty DataFrame
Columns: []
Index: []


The script should return an empty data frame. This indicates that there are no row pairs for which BG pairs in which total population is zero but percent 2.0 times the poverty line is non-zero. If it is not empty, ensure that the correct file is being used as the input file. If it is empty, please continue.



We can now remove all of the zero population BGs from our csv file. Let's start by printing them and saving them to a seperate csv file called "deleted_zero_pop_bgs.csv" in case we want to reference them again.

In [117]:
#order = 1 (total population) has an estimate that equals zero, AND order = 8 (population > 2.0 times the poverty line) estimate equals zero

import pandas as pd

# Load the CSV file (same input file as previous code box)
input_file = f"{ipath}/{'edited_VRE_complete_NJ.csv'}"  # Replace with your CSV file name
data = pd.read_csv(input_file)

# Load the CSV file
data = pd.read_csv(input_file)

# Initialize a list to store the identified pairs
identified_pairs = []

# Iterate over the dataframe, processing each pair of rows
for i in range(1, data.shape[0], 2):
    if data.iloc[i-1, 1] == 1 and data.iloc[i, 1] == 8 and data.iloc[i-1, 0] == data.iloc[i, 0]:
        # Extract the GEOID for the current pair
        geoid = data.iloc[i-1, 0]
        
        # Extract the estimates for the current pair
        estimate_order_1 = data.iloc[i-1]['ESTIMATE']
        estimate_order_8 = data.iloc[i]['ESTIMATE']
        
        # Check if both ORDER = 1 and ORDER = 8 estimates are zero
        if estimate_order_1 == 0 and estimate_order_8 == 0:
            # Append the rows to the identified_pairs list
            identified_pairs.append(data.iloc[i-1, :])
            identified_pairs.append(data.iloc[i, :])

# Convert the list of identified pairs to a DataFrame for easier visualization
zero_pop_df = pd.DataFrame(identified_pairs)

# Define df as data frame of identified pairs only
df = zero_pop_df

# Print number of identified pairs
num_rows = df.shape[0]
print("Number of zero population block groups")

# Print "num_rows" divided by two since each pair of rows represents 1 block group
print(num_rows // 2)

# Store this value for later use
num_zero_pop = num_rows // 2

# Print the identified pairs
print(zero_pop_df)

# Define the output file path
output_file = f"{opath}/deleted_zero_pop_bgs.csv"

# Write the identified pairs to a CSV file
df.to_csv(output_file, index=False)

print(f"Results have been written to {output_file}")

Number of zero population block groups
90
                       GEOID  ORDER  ESTIMATE  Var_Rep1  Var_Rep2  Var_Rep3  \
386    1500000US340019900000      1         0         0         0         0   
387    1500000US340019900000      8         0         0         0         0   
1974   1500000US340057001043      1         0         0         0         0   
1975   1500000US340057001043      8         0         0         0         0   
1986   1500000US340057003034      1         0         0         0         0   
...                      ...    ...       ...       ...       ...       ...   
10725  1500000US340299801001      8         0         0         0         0   
10726  1500000US340299900000      1         0         0         0         0   
10727  1500000US340299900000      8         0         0         0         0   
11576  1500000US340339900000      1         0         0         0         0   
11577  1500000US340339900000      8         0         0         0         0   

       Va

 Now let's actually remove all of the zero population BGs from our csv file. The output of the following script is a new csv file titled "filtered_complete_NJ.csv" that contains all of the original BGs except for the zero population BGs identified above. 

In [118]:
#Create new csv file without zero pop BGs

import pandas as pd

# Load the CSV file (same input file as previous code box)
input_file = f"{ipath}/{'edited_VRE_complete_NJ.csv'}"  # Replace with your CSV file name
data = pd.read_csv(input_file)

# Define the estimate column
estimate_column = 'ESTIMATE'  # Replace with the actual column name if different

# Identify rows with ORDER = 8 and Estimate = 0
rows_with_order_8_zero = data[(data['ORDER'] == 8) & (data[estimate_column] == 0)]

# Identify rows with ORDER = 1 and Estimate = 0
rows_with_order_1_zero = data[(data['ORDER'] == 1) & (data[estimate_column] == 0)]

# Get the block group identifiers for these rows
block_group_order_8_zero = set(rows_with_order_8_zero['GEOID'])
block_group_order_1_zero = set(rows_with_order_1_zero['GEOID'])

# Find common block groups where both ORDER = 8 and ORDER = 1 have zero estimates
common_block_groups = block_group_order_8_zero.intersection(block_group_order_1_zero)

# Filter out rows with these common block groups
filtered_data = data[~data['GEOID'].isin(common_block_groups)]

# Define the output file path
output_file = f"{opath}/filtered_complete_NJ.csv"

# Write the filtered DataFrame to a new CSV file
filtered_data.to_csv(output_file, index=False)

data = pd.read_csv(output_file)

# Calculate the number of block groups in the input file
num_filtered_data = data.shape[0]//2
print("Number of block groups in filtered_complete_NJ.csv", num_filtered_data)

print(f"Filtered results have been written to {output_file}")

#Check to make sure that the number of block groups in the new csv file is equal to the original csv file - the number of zero pop block groups

if num_filtered_data == num_original - num_zero_pop:
    print("The number of block groups matches after subtraction.")
else:
    print("There is a discrepancy in the number of block groups after subtraction.")


Number of block groups in filtered_complete_NJ.csv 6509
Filtered results have been written to /Users/ayla/workflow/data/filtered_complete_NJ.csv
The number of block groups matches after subtraction.


Now make sure that 100% low income block groups weren't deleted.

In [3]:
import pandas as pd

main_filename = "filtered_complete_NJ.csv"
subset_filename = "100%_low_income_bgs.csv"

# Load the CSV files
main_file = pd.read_csv(f"{ipath}/{main_filename}")
subset_file = pd.read_csv(f"{ipath}/{subset_filename}")

# Check if all rows in the subset file are present in the main file
merged = subset_file.merge(main_file, how='left', indicator=True)
missing_rows = merged[merged['_merge'] == 'left_only']

if missing_rows.empty:
    print("All rows in the subset file are present in the main file.")
else:
    print("Some rows in the subset file are not present in the main file.")
    print("The following rows in the subset file are not present in the main file:")
    print(missing_rows.drop(columns='_merge'))

All rows in the subset file are present in the main file.


Check for zero values in the variance replicate estimates of the remaining "total population" rows. Zero values will cause NaN values in the later confidence interval script. 

In [10]:
#check for zero values in replicates of remaining total population rows
import pandas as pd

# Define the file path
path = os.path.join(os.path.dirname(os.getcwd()))
path='/Users/ayla/workflow'
ipath = os.path.join(path,'data')
filename = "filtered_complete_NJ.csv"
csv_file_path = f"{ipath}/{filename}"

# Load the CSV file
data = pd.read_csv(csv_file_path)

# Filter rows where Order = 1
order_1_rows = data[data['ORDER'] == 1]

# Further filter rows where any column has a zero value
rows_with_zero_values = order_1_rows[(order_1_rows == 0).any(axis=1)]

# Print the identified rows
print("Rows where Order = 1 and any column has a zero value:")
print(rows_with_zero_values)

Rows where Order = 1 and any column has a zero value:
                      GEOID  ORDER  ESTIMATE  Var_Rep1  Var_Rep2  Var_Rep3  \
5404  1500000US340155020026      1        11        10         1        24   
5430  1500000US340155024001      1         4        15         0         1   

      Var_Rep4  Var_Rep5  Var_Rep6  Var_Rep7  ...  Var_Rep71  Var_Rep72  \
5404        15        10         9        17  ...         22         20   
5430        10         1         4         5  ...          8          4   

      Var_Rep73  Var_Rep74  Var_Rep75  Var_Rep76  Var_Rep77  Var_Rep78  \
5404         15         11         22          1         10          1   
5430          4          0          3          6         13          4   

      Var_Rep79  Var_Rep80  
5404          1         16  
5430          2          3  

[2 rows x 83 columns]
